# 1. 데이터 로드

In [ ]:
!pip install backtrader
!pip install backtesting

     |████████████████████████████████| 410 kB 5.4 MB/s 
     |████████████████████████████████| 175 kB 4.7 MB/s 
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173822 sha256=687313d2c1a3483cfc412cf11e9bf18fc8235c44039d910ebb9ff956c0f50415
  Stored in directory: /root/.cache/pip/wheels/48/1d/ea/65dacebc37da7655d8a1fd0f315ac39d102e31d6545237a1c6
Successfully built backtesting


In [ ]:
import numpy as np
from google.colab import drive
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn import metrics
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Activation, Flatten, Dropout
from keras.layers import LSTM, SimpleRNN, GRU, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Concatenate
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import backtrader as bt
from backtrader.feeds import PandasData
import backtrader.indicators as btind

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# 2. 데이터 전처리

In [ ]:
def datasplit(df, Y_colname, X_colname): #데이터 분리 함수
  df_train = df.loc[0 : round(len(df) * 0.75)] 
  df_test = df.loc[round(len(df) * 0.75) + 1 : ]
  Y_train = df_train[Y_colname]
  X_train = df_train[X_colname]
  Y_test = df_test[Y_colname]
  X_test = df_test[X_colname]
  return X_train, X_test, Y_train, Y_test

In [ ]:
def get_preprocessed_data(data, seq_length=30):
  #최신 데이터가 뒤로 오도록 정렬
  data = data.sort_values(by=['TRD_DD'])
  data.set_index('TRD_DD',drop=True,inplace=True)
  data.reset_index('TRD_DD',drop=False,inplace=True)

  #X/Y데이터로 분리
  Y_colname = ['TREND'] #종속변수
  X_remove = ['TRD_DD']
  X_colname = [x for x in data.columns if x not in Y_colname + X_remove] #입력변수들

  #train/test로 분리
  X_train, X_test, Y_train, Y_test = datasplit(data, Y_colname, X_colname)

  #array로 변환
  X_train_arr = X_train.values
  Y_train_arr = Y_train.values
  X_test_arr = X_test.values
  Y_test_arr = Y_test.values

  #sequence 생성
  X_train, Y_train = [], []
  for index in range(len(X_train_arr) - seq_length): #sequence개의 데이터만큼 차분데이터 분석에 사용하므로, 총 데이터 개수 - sequence개수가 전체 데이터의 길이 --> 총길이만큼 돌리는 것이므로, 기존 차원을 뜻하는 (row, column) 꼴에는 변함 없음 --> 2차원 데이터
    X_train.append(np.array(X_train_arr[index:index + seq_length])) #X_train 데이터 --> index가 sequence인 1차원 데이터 생성 --> sequence를 뜻하는 차원 생성
    Y_train.append(np.ravel(Y_train_arr[index + seq_length:index + seq_length + 1])) #Y_train 데이터 --> index가 sequence인 1차원 데이터 생성 --> 그러나 ravel() 로 차원축소 --> sequence차원 생성되지 않음.
  X_train, Y_train = np.array(X_train), np.array(Y_train) #array로 저장되어있는 리스트를 array로 변환해서 저장
  X_test, Y_test = [], [] #동일한 과정 test데이터셋에도 적용
  for index in range(len(X_test_arr) - seq_length):
    X_test.append(np.array(X_test_arr[index:index + seq_length])) 
    Y_test.append(np.ravel(Y_test_arr[index + seq_length:index + seq_length + 1]))
  X_test, Y_test = np.array(X_test), np.array(Y_test)

  return X_train, Y_train, X_test, Y_test

# 3. 모델링

In [ ]:
batch_size = 60 #미니 배치 내 데이터 수
epoch = 80 #돌릴 횟수

In [ ]:
def Inception_sim_model(): 
  input_tensor = Input(shape=(X_train.shape[1], X_train.shape[2])) # 30 * 5 (sequence * channel)

  f10_layer = Conv1D(256, 1, 1, activation='relu')(input_tensor) # 30 * 256
  f10_layer_1 = Conv1D(128, 10, 1, activation='relu')(f10_layer) # 21 * 128 
  f10_layer_2 = Conv1D(64, 6, 1, activation='relu')(f10_layer_1) # 16 * 64
  f10_layer_3 = Conv1D(32, 3, 1, activation='relu')(f10_layer_2) # 14 * 32

  f15_layer = Conv1D(256, 1, 1, activation='relu')(input_tensor) # 30 * 256
  f15_layer_1 = Conv1D(128, 15, 1, activation='relu')(f15_layer) # 16 * 128
  f15_layer_2 = Conv1D(64, 3, 1, activation='relu')(f15_layer_1) # 14 * 64

  f7_layer = Conv1D(256, 1, 1, activation='relu')(input_tensor) # 30 * 256
  f7_layer_1 = Conv1D(128, 7, 1, activation='relu')(f7_layer) # 24 * 128
  f7_layer_2 = Conv1D(64, 7, 1, activation='relu')(f7_layer_1) # 18 * 64
  f7_layer_3 = Conv1D(32, 5, 1, activation='relu')(f7_layer_2) # 14 * 32

  f2_pooling = MaxPooling1D(2)(input_tensor) # 15 * 5
  f2_layer = Conv1D(128, 2, 1, activation='relu')(f2_pooling) # 14 * 128

  concat_layer = Concatenate(axis=-1)([f10_layer_3, f15_layer_2, f7_layer_3, f2_layer]) # 14 * 256

  gru_layer = GRU(512, return_sequences=True, dropout=0.2, activation='relu')(concat_layer) # 14 * 512
  gru_layer_1 = GRU(256, return_sequences=True, dropout=0.2, activation='relu')(gru_layer) # 14 * 256
  gru_layer_2 = GRU(64, return_sequences=False, dropout=0.2, activation='relu')(gru_layer_1) # 64
  output = Dense(1)(gru_layer_2) # 1

  model = Model(inputs=input_tensor, outputs=output)
  return model

In [ ]:
def GRU_model():
  input_tensor = Input(shape=(X_train.shape[1], X_train.shape[2])) # 17 * 5
  x = GRU(128, return_sequences=True, dropout=0.2, activation='relu')(input_tensor) # 17 * 128
  x = GRU(256, return_sequences=True, dropout=0.2, activation='relu')(x) # 17 * 256
  x = GRU(64, return_sequences=False, dropout=0.2, activation='relu')(x) # 64
  output = Dense(1)(x) # 1
  model = Model(inputs=input_tensor, outputs=output)
  return model

In [ ]:
def LSTM_model():
  input_tensor = Input(shape=(X_train.shape[1], X_train.shape[2])) # 17 * 5
  x = LSTM(128, return_sequences=True, dropout=0.2, activation='relu')(input_tensor) # 17 * 128
  x = LSTM(256, return_sequences=True, dropout=0.2, activation='relu')(x) # 17 * 256
  x = LSTM(64, return_sequences=False, dropout=0.2, activation='relu')(x) # 64
  output = Dense(1)(x) # 1
  model = Model(inputs=input_tensor, outputs=output)
  return model

In [ ]:
def train_model(X_train, Y_train, X_test, Y_test, modelname='Inception_sim_model'): #모델 학습 함수
  if modelname == 'Inception_sim_model':
    model = Inception_sim_model()
  elif modelname == 'CNN_GRU_model':
    model = CNN_GRU_model()
  elif modelname == 'LSTM_model':
    model = LSTM_model()
  elif modelname == 'GRU_model':
    model = GRU_model() 

  model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
  rlr_cb = ReduceLROnPlateau(patience=5, mode='min')
  ely_cb = EarlyStopping(patience=10, mode='min') 
  hist = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epoch, verbose=1, validation_split=0.15, callbacks=([rlr_cb, ely_cb]))

  return model, hist

# 4. backtest

In [ ]:
class PandasData_Signal(PandasData):

  lines = ('signal',)

  # add the parameter to the parameters inherited from the base class

  params = (('signal', 6),)

class MLSignal(bt.SignalStrategy):

    def log(self, txt, dt=None):

        ''' Logging function fot this strategy'''

        dt = dt or self.datas[0].datetime.date(0)

        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.signal = self.datas[0].signal
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                    order.executed.value,
                    order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                (trade.pnl, trade.pnlcomm))
        
    def next(self):
        #Check if we are in the market 
        if not self.position :            
            if self.signal[0] == -1:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                size = int(self.broker.getcash() / self.dataclose[0])
                #Keep track of the created order to avoid a 2nd order
                self.order = self.buy(size = size)
                
        else:           
            #Already in the market...we might sell                     
            if self.signal[0] == 1:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

# 실행

In [ ]:
mktcap_top = ['KR7005930003', 'KR7000660001', 'KR7035420009', 'KR7207940008', 'KR7005931001', 'KR7035720002', 'KR7006400006', 'KR7005380001', 'KR7051910008', 'KR7000270009'] # 데이터 로드

In [ ]:
# 실행 코드
final_df = pd.DataFrame([], columns = ['full_code', 'train', 'test', 'MDD', 'CAGR', 'profit'])

for i in range(len(mktcap_top)):
  mktcap_data = pd.read_csv('/content/drive/MyDrive/data/mktcap10/{}.csv'.format(mktcap_top[i]))
  mktcap_data['TREND'] = mktcap_data['TREND'].fillna(0)

  X_train, Y_train, X_test, Y_test = get_preprocessed_data(mktcap_data, 30)

  optimum_df = pd.DataFrame([], columns = ['train', 'test', 'MDD', 'CAGR', 'profit'])
  
  headers = ['train', 'test', 'MDD', 'CAGR', 'profit']
  profit_df = pd.DataFrame([], columns = headers)

  model, hist = train_model(X_train, Y_train, X_test, Y_test, modelname='GRU_model')  # modelname='LSTM_model' 로 수정시 LSTM모델, 'CNN_GRU_model' 로 수정시 CNN_GRU 모델로 바뀜

  past = model.predict(X_test)

  Y_train_pred = model.predict(X_train)
  Y_test_pred = model.predict(X_test)


  a = list(X_train.shape)
  b = list(X_test.shape)
  test_data = mktcap_data.iloc[a[0]:]

  test_score = test_data.copy()
  test_score.reset_index(drop=True, inplace=True)
  imsi_df = pd.DataFrame(Y_test_pred, columns=['pred'])
  test_score = pd.concat([test_score, imsi_df], axis=1)
  test_score.fillna(0, inplace=True)
  test_score = test_score[['TRD_DD', 'pred']]
  test_score = test_score.reset_index()
  test_score = test_score.drop(['index'], axis = 1)

  for j in range(len(test_score)):
    if test_score['pred'][j] < -0.5:
      test_score['pred'][j] = -1
    elif test_score['pred'][j] > 0.5:
      test_score['pred'][j] = 1
    else:
      test_score['pred'][j] = 0
  print('\n', test_score.value_counts(['pred']), '--------------------------------')

  json_df = pd.read_csv('/content/drive/MyDrive/data/raw_data/{}.csv'.format(mktcap_top[i])) # 첫번째 키값으로 데이터프레임 변환
  json_df = json_df[['TRD_DD', 'TDD_OPNPRC', 'TDD_HGPRC', 'TDD_LWPRC', 'TDD_CLSPRC', 'ACC_TRDVOL']]
  json_df = json_df.sort_values(by=['TRD_DD'])
  json_df.reset_index(drop=True,inplace=True)

  df_end = json_df.index[json_df['TRD_DD'] == '2021/12/30'][0]
  raw = json_df.iloc[a[0]:df_end+1]
  raw.reset_index(drop = True, inplace = True)
  raw['TRD_DD'] = pd.to_datetime(raw['TRD_DD'])
                  
  final = pd.merge(raw, test_score, left_index = True, right_index = True).drop(columns='TRD_DD_y')
  final = final.rename(columns={'TRD_DD_x': 'TRD_DD'})
  final['TDD_OPNPRC'] = final['TDD_OPNPRC'].str.replace(',','').astype('float')
  final['TDD_HGPRC'] = final['TDD_HGPRC'].str.replace(',','').astype('float')
  final['TDD_LWPRC'] = final['TDD_LWPRC'].str.replace(',','').astype('float')
  final['TDD_CLSPRC'] = final['TDD_CLSPRC'].str.replace(',','').astype('float')
  final['ACC_TRDVOL'] = final['ACC_TRDVOL'].str.replace(',','').astype('float')
  final = final.astype({'pred':'int'})

  # 세레브로 (벡트레이더 엔진) 설정

  # 세레브로 가져오기
  cerebro = bt.Cerebro()

  # 데이터 가져오기
  data = PandasData_Signal(dataname = final,
                                  datetime = 0,
                                  open = 1,
                                  high = 2,
                                  low = 3,
                                  close = 4,
                                  volume = 5,
                                  openinterest = -1,
                                  signal = 6,
                                  )

  # 데이터 추가하기
  cerebro.adddata(data)

  # 전략 추가하기
  cerebro.addstrategy(MLSignal)

  # 브로커 설정
  cerebro.broker.setcash(10000000)

  # 매매 단위 설정
  #cerebro.addsizer(bt.sizers.SizerFix, stake = 30) #한번에 30주 설정.

  # 3. 세레브로 실행하기

  # 초기 투자금 가져오기
  init_cash = cerebro.broker.getvalue()

  # 평가지표 추가
  cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
  cerebro.addanalyzer(bt.analyzers.Returns)
  cerebro.addanalyzer(bt.analyzers.DrawDown)

  # 세레브로 실행하기
  results = cerebro.run()

  # 최종 금액 가져오기
  final_cash = cerebro.broker.getvalue()

  analyzers = results[0].analyzers
  mdd = analyzers.drawdown.get_analysis()['max']['drawdown']
  cagr = analyzers.returns.get_analysis()['rnorm100']
  # sharp = analyzers.sharperatio.get_analysis()['sharperatio']
  print(f"mdd : {mdd:.2f}, cagr: {cagr:.2f}")


  profit = float(final_cash - init_cash)/float(init_cash) * 100.
  print("최종금액 : ", final_cash, "원")
  print("수익률 : ", profit, "%")
  print("\n")

  profit_list = [a[0], b[0], mdd, cagr, profit]
  profit_df = profit_df.append(pd.Series(profit_list, index = headers), ignore_index = True)

  max_profit = profit_df[profit_df['profit'] == profit_df['profit'].max()].reset_index(drop = True)


  num_max_profit = pd.concat([max_profit], axis = 1)
  optimum_df = pd.concat([optimum_df, num_max_profit], axis = 0).reset_index(drop = True)
  # print(optimum_df)

  op_list = optimum_df.iloc[0].tolist()
  op_list.insert(0, mktcap_top[i])
  print(op_list)

  final_df = final_df.append(pd.Series(op_list, index=final_df.columns), ignore_index=True)

print(final_df)

Epoch 1/80
72/72 [==============================] - 33s 378ms/step - loss: 0.3160 - accuracy: 0.0110 - val_loss: 0.2723 - val_accuracy: 0.0079 - lr: 0.0010
Epoch 2/80
72/72 [==============================] - 27s 380ms/step - loss: 0.2830 - accuracy: 0.0112 - val_loss: 0.2785 - val_accuracy: 0.0079 - lr: 0.0010
Epoch 3/80
72/72 [==============================] - 31s 437ms/step - loss: 0.2788 - accuracy: 0.0112 - val_loss: 0.2579 - val_accuracy: 0.0079 - lr: 0.0010
Epoch 4/80
72/72 [==============================] - 30s 424ms/step - loss: 0.2692 - accuracy: 0.0114 - val_loss: 0.2605 - val_accuracy: 0.0079 - lr: 0.0010
Epoch 5/80
72/72 [==============================] - 27s 371ms/step - loss: 0.2676 - accuracy: 0.0121 - val_loss: 0.2582 - val_accuracy: 0.0092 - lr: 0.0010
Epoch 6/80
72/72 [==============================] - 27s 375ms/step - loss: 0.2648 - accuracy: 0.0110 - val_loss: 0.2732 - val_accuracy: 0.0092 - lr: 0.0010
Epoch 7/80
72/72 [==============================] - 27s 367ms/st

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



 pred
 0.0    1582
 1.0     101
-1.0      36
dtype: int64 --------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2015-10-08, BUY CREATE, 1270000.00
2015-10-12, BUY EXECUTED, Price: 1260000.00, Cost: 8820000.00, Comm 0.00
2016-05-26, SELL CREATE, 1296000.00
2016-05-27, SELL EXECUTED, Price: 1300000.00, Cost: 1260000.00, Comm 0.00
2016-05-27, SELL CREATE, 1282000.00
2016-05-30, SELL EXECUTED, Price: 1297000.00, Cost: 1260000.00, Comm 0.00
2016-05-30, SELL CREATE, 1280000.00
2016-05-31, SELL EXECUTED, Price: 1280000.00, Cost: 1260000.00, Comm 0.00
2016-05-31, SELL CREATE, 1292000.00
2016-06-01, SELL EXECUTED, Price: 1298000.00, Cost: 1260000.00, Comm 0.00
2016-06-01, SELL CREATE, 1333000.00
2016-06-02, SELL EXECUTED, Price: 1350000.00, Cost: 1260000.00, Comm 0.00
2017-04-24, SELL CREATE, 2062000.00
2017-04-25, SELL EXECUTED, Price: 2073000.00, Cost: 1260000.00, Comm 0.00
2017-04-25, SELL CREATE, 2135000.00
2017-04-26, SELL EXECUTED, Price: 2135000.00, Cost: 1260000.00, Comm 0.00
2017-04-26, OPERATION PROFIT, GROSS 1913000.00, NET 1913000.00
2017-05-17, BUY CREATE, 2317000.00
2017-05-18, BUY EXECUTED

In [ ]:
gru

,full_code,train,test,MDD,CAGR,profit
0,KR7005930003,5043.0,1659.0,30.483228,10.068002,91.8805
1,KR7000660001,4677.0,1537.0,36.924358,23.399589,276.8305
2,KR7035420009,3534.0,1157.0,74.028569,-12.058934,-46.0450
3,KR7207940008,925.0,287.0,21.327014,7.232873,9.8800
4,KR7005931001,5043.0,1659.0,95.405920,-28.762647,-90.0150
5,KR7035720002,4076.0,1337.0,31.532944,7.384214,48.1370
6,KR7006400006,5043.0,1659.0,37.243421,11.316768,107.1650
7,KR7005380001,5043.0,1659.0,36.225000,1.396933,9.8830
8,KR7051910008,3813.0,1249.0,40.543262,15.424753,109.8350
9,KR7000270009,5043.0,1659.0,56.135498,6.530752,53.6935


In [ ]:
gru_cnn #gru +cnn

,full_code,train,test,MDD,CAGR,profit
0,KR7005930003,5043.0,1659.0,27.651083,8.588397,75.0250
1,KR7000660001,4677.0,1537.0,29.749461,16.200274,157.8850
2,KR7035420009,3534.0,1157.0,83.324198,-27.429508,-78.5500
3,KR7207940008,925.0,287.0,13.997919,-6.167203,-8.2300
4,KR7005931001,5043.0,1659.0,36.617789,9.017021,79.7725
5,KR7035720002,4076.0,1337.0,40.387936,14.226317,108.2720
6,KR7006400006,5043.0,1659.0,30.905241,12.748483,125.9550
7,KR7005380001,5043.0,1659.0,35.713045,4.539795,35.2050
8,KR7051910008,3813.0,1249.0,31.772059,3.011988,16.5700
9,KR7000270009,5043.0,1659.0,42.792745,13.055889,130.1735
